In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from time import sleep
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
import pandas as pd

# ヘッドレスモードの設定
options = Options()
options.add_argument('--headless')

# WebDriverの起動
service = Service('chromedriver.exe')
browser = webdriver.Chrome(service=service,options=options)

# 開始URL
url = 'https://filmarks.com/list-anime/vod/netflix'
browser.get(url)
sleep(3)

# ページURLを格納するリスト
HREF_LIST = []
anime_titles_elements = []
anime_scores_elements = []
anime_companies_elements = []
anime_voice_elements = []
anime_songs_elements = []
anime_stories_elements = []
anime_watches_elements = []

i = 0

# ページ巡回
while True:
    elements = browser.find_elements(By.CSS_SELECTOR, 'div.p-content-cassette__people__readmore a')
    for element in elements:
        href = element.get_attribute('href')
        if href:
            HREF_LIST.append(href)
    print(HREF_LIST)
    # 次ページボタン処理
    try:
        next_button = WebDriverWait(browser, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a.c2-pagination__next"))
        )
        browser.execute_script("arguments[0].scrollIntoView();", next_button)
        sleep(1)
        next_button.click()
        i += 1
        if i == 5:  # 5ページまで取得
            break
        sleep(2)
    except Exception as e:
        print("次のページがないかボタンがクリックできません:", e)
        break

# アニメ情報の取得
for EACH_LIST in HREF_LIST:
    browser.get(EACH_LIST)
    sleep(3)
    try:
        # 初期化
        anime_voice = []
        anime_song = []
        
        anime_titles = browser.find_elements(By.CSS_SELECTOR, 'h2.p-content-detail__title')
        anime_scores = browser.find_elements(By.CSS_SELECTOR, '.c-rating__score')
        anime_watchies = browser.find_elements(By.CSS_SELECTOR, 'span.c-content__count')
        anime_stories = browser.find_elements(By.CSS_SELECTOR, '.p-content-detail__synopsis-desc.is-show-all')
        anime_companies = browser.find_elements(By.CSS_SELECTOR, '.p-content-detail__other-info ul li a')
        
        num_items = min(len(anime_titles), len(anime_scores), len(anime_watchies),len(anime_stories),len(anime_companies))
        
        for i in range(num_items):
            anime_title = anime_titles[i].text if i < len(anime_titles) else "タイトルなし"
            anime_score = anime_scores[i].text if i < len(anime_scores) else "スコアなし"
            anime_watch = anime_watchies[i].text if i < len(anime_watchies) else "視聴回数なし"
            anime_company = anime_companies[i].text if i < len(anime_companies) else "情報なし"
            
            # あらすじの確認
            if i < len(anime_stories):
                anime_story = anime_stories[i].text
            else:
                anime_story = "あらすじなし"
       
        # 声優
        artists = browser.find_elements(By.CSS_SELECTOR, '.c2-button-tertiary-s-multi-text__text')
        anime_voice = [artist.text for artist in artists if artist.text]

        # 主題歌・挿入歌
        serch = browser.find_elements(By.CSS_SELECTOR, '.p-content-detail__people-list-others')
        if serch:
            songs = serch[-1].find_elements(By.CSS_SELECTOR, 'ul li a')
            anime_song = [song.text for song in songs if song.text]


#リストに格納
        anime_titles_elements.append(anime_title)
        anime_scores_elements.append(anime_score)
        anime_watches_elements.append(anime_watch)
        anime_companies_elements.append(anime_company)
        anime_voice_elements.append(anime_voice)
        anime_songs_elements.append(anime_song)
        anime_stories_elements.append(anime_story)
    except Exception as e:
        print("[ERROR] 情報取得中にエラー:", e)
        continue

# ブラウザを閉じる
browser.quit()

#データの格納
df = pd.DataFrame()
df['タイトル名'] = anime_titles_elements
df['評価'] = anime_scores_elements
df['視聴数'] = anime_watches_elements
df['制作会社'] = anime_companies_elements
df['声優'] = anime_voice_elements
df['主題歌・挿入歌'] = anime_songs_elements
df['あらすじ'] = anime_stories_elements

df.to_csv('アニメ情報.csv',index = False)

['https://filmarks.com/animes/4335/5840?vod=5', 'https://filmarks.com/animes/4523/6116?vod=5', 'https://filmarks.com/animes/3613/5629?vod=5', 'https://filmarks.com/animes/4333/5836?vod=5', 'https://filmarks.com/animes/3964/5360?vod=5', 'https://filmarks.com/animes/3844/5200?vod=5', 'https://filmarks.com/animes/4516/6109?vod=5', 'https://filmarks.com/animes/4315/5811?vod=5', 'https://filmarks.com/animes/860/632?vod=5', 'https://filmarks.com/animes/47/72?vod=5', 'https://filmarks.com/animes/3865/5222?vod=5', 'https://filmarks.com/animes/3131/5424?vod=5', 'https://filmarks.com/animes/3613/4883?vod=5', 'https://filmarks.com/animes/23/41?vod=5', 'https://filmarks.com/animes/3303/4233?vod=5', 'https://filmarks.com/animes/314/467?vod=5', 'https://filmarks.com/animes/342/5430?vod=5', 'https://filmarks.com/animes/3991/5399?vod=5', 'https://filmarks.com/animes/2/3?vod=5', 'https://filmarks.com/animes/2266/2917?vod=5', 'https://filmarks.com/animes/1304/1731?vod=5', 'https://filmarks.com/animes/86

KeyboardInterrupt: 